# Loading Libraries

In [5]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import datetime

In [6]:
#movies_data=pd.read_csv(r"/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv")
#credits_data=pd.read_csv(r"/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv")
movies_data=pd.read_csv(r"D:\Projects\Content-Based_Movie_Recommendation_System-main\Backend\Data\tmdb_5000_movies.csv")
credits_data=pd.read_csv(r"D:\Projects\Content-Based_Movie_Recommendation_System-main\Backend\Data\tmdb_5000_credits.csv")


In [7]:
movies_data.head()
movies_data.columns 
# 'budget','genres','homepage','id','keywords','original_language','original_title','overview','popularity','production_companies','production_countries','release_date','revenue','runtime','spoken_languages','status','tagline','title','vote_average','vote_count'
movies_data.shape  # 4803,20
print(movies_data.isna().sum())


budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64


In [8]:
credits_data.head()
credits_data.columns # 'movie_id', 'title', 'cast', 'crew'
credits_data.shape  # 4803,4
print(credits_data.isna().sum())

movie_id    0
title       0
cast        0
crew        0
dtype: int64


# Merging 2 data-sets: 

In [9]:
movies=movies_data
movies=movies.merge(credits_data,on="title")    # As "title" column present on both data-set, so mergeing on"title"
movies.shape   # (4809, 23)

(4809, 23)

## Data-set insights

In [10]:
movies.head()
movies.shape   # 4809,8

# handling Null values
movies.isna().sum()
#homepage: 3096 ; overview: 3 ; release_date: 1 ; runtime: 2 ; tagline: 844

# Duplicate values
movies.duplicated().sum()

0

In [11]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [12]:
def get_unique_value(df):
    """ Takes Column & returns set(unique items present) """
    to_list=df.tolist()
    unique_items=set()

    for each_list in to_list:
        for item in each_list:
            unique_items.add(item)
    return unique_items 

# genres_unique_list=get_unique_value(movies["genres"])
# genres_unique_list

## Preprocessing Data(Column-wise)

In [13]:
#'movie_id' column
movies['movie_id'].head()
movies['movie_id'][0]  

# we no need to preprocess this column

19995

In [14]:
#'title' column
movies['title'].head()
movies['title'][0]  

# we no need to preprocess this column

'Avatar'

### 'genres' column

In [15]:
movies['genres'].head()

movies['genres'].isna().sum()   # 0
movies['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [16]:
## text-preprocessing

# to make all words lower-case
movies["genres"]=movies["genres"].str.lower()

# Here we only need "names" from "genres"
# we also need to remove " " As 'Science','Fiction' & 'Science Fiction' can be represent as different 
def genre_extractor(df):
    genres=[i["name"].replace(" ","-") for i in eval(df)]    # extract only "name"s from "genre-column"
    return genres

movies["genres"]=movies["genres"].apply(genre_extractor)
movies["genres"]

0       [action, adventure, fantasy, science-fiction]
1                        [adventure, fantasy, action]
2                          [action, adventure, crime]
3                    [action, crime, drama, thriller]
4                [action, adventure, science-fiction]
                            ...                      
4804                        [action, crime, thriller]
4805                                [comedy, romance]
4806               [comedy, drama, romance, tv-movie]
4807                                               []
4808                                    [documentary]
Name: genres, Length: 4809, dtype: object

In [17]:
## Unique values 
genres_unique_list=get_unique_value(movies["genres"])
# {'action','adventure','animation','comedy','crime','documentary','drama','family','fantasy','foreign',
# 'history','horror','music','mystery','romance','science-fiction','thriller','tv-movie','war','western'}

### 'keywords' column

In [18]:
movies['keywords'].head()
movies['keywords'][0]
movies['keywords'].isna().sum()

0

In [19]:
## text-preprocessing

# to make all words lower-case
movies["keywords"]=movies["keywords"].str.lower()

# Here we only need "names" from "key-words"
# we also need to remove " " As 'Science','Fiction' & 'Science Fiction' can be represent as different  
def keywords_extractor(df):
    keywords=[i["name"].replace(" ","-") for i in eval(df)]    # extract only "name"s from "keywords-column"
    return keywords

movies["keywords"]=movies["keywords"].apply(keywords_extractor)
movies["keywords"]

0       [culture-clash, future, space-war, space-colon...
1       [ocean, drug-abuse, exotic-island, east-india-...
2       [spy, based-on-novel, secret-agent, sequel, mi...
3       [dc-comics, crime-fighter, terrorist, secret-i...
4       [based-on-novel, mars, medallion, space-travel...
                              ...                        
4804    [united-states–mexico-barrier, legs, arms, pap...
4805                                                   []
4806    [date, love-at-first-sight, narration, investi...
4807                                                   []
4808            [obsession, camcorder, crush, dream-girl]
Name: keywords, Length: 4809, dtype: object

In [20]:
## Unique values 
get_unique_value(movies["keywords"])

{'fictionalized-biography',
 'community-service',
 'infestation',
 'friends-who-hate-each-other',
 'kendo',
 'pink-bathrobe',
 'tonto',
 'education',
 'african-american',
 'music-tour',
 'torpedo',
 'subway',
 'figaro',
 'internment-camp',
 'master-warrior',
 'patriarch',
 'exploding-ship',
 "life's-dream",
 'physicist',
 'violence-against-women',
 'swordplay',
 'space-marine',
 'golden-gun',
 'seasickness',
 'christmas-party',
 'half-breed',
 'wrongful-conviction',
 'vengeful',
 'postwar-period',
 'drug-humor',
 'laundry',
 'first-time',
 'panzer',
 'gala',
 'lighthouse',
 'paris-france',
 'psychological-stress',
 'compulsive-gambling',
 'alternate-timeline',
 'prison',
 'overboard',
 'eastern-block',
 'stretch-limousine',
 'midnight',
 'totalitarian',
 'loser',
 'blood-ritual',
 "frankenstein's-monster",
 'fighter',
 'rock-opera',
 'basement-garage',
 'artificial',
 'death-of-superhero',
 'music-video',
 'space-carrier',
 'newcastle-united',
 'belgium',
 'governor',
 'year-2002',
 'c

### 'overview'

In [21]:
movies["overview"].head()
movies["overview"][0]

movies["overview"].isna().sum()   # 3
movies["overview"].fillna(" ",inplace=True)

In [22]:
# make all words lower-case
movies["overview"]=movies["overview"].str.lower()

# Remove "punctuation" 
movies["overview"]=movies['overview'].str.replace('[^\w\s]','')

# Remove "stop-words" as those are not important 
# Also need to "Stem" these words , As "play" , "plays" & "playing" can be represent as different.
stop_words=stopwords.words('english')
ps=PorterStemmer()

def preprocess(words):
    preprocessed=[word for word in words.split() if word not in stop_words]  # removoing stop-words
    preprocessed=[ps.stem(word) for word in preprocessed]                   # steming 
    preprocessed=' '.join(preprocessed) # join words to sentence
    return preprocessed

movies['preprocessed_overview']=movies['overview'].apply(preprocess)
movies['preprocessed_overview']

<ipython-input-22-e7850b1e2416>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  movies["overview"]=movies['overview'].str.replace('[^\w\s]','')


0       22nd centuri parapleg marin dispatch moon pand...
1       captain barbossa long believ dead come back li...
2       cryptic messag bond past send trail uncov sini...
3       follow death district attorney harvey dent bat...
4       john carter warweari former militari captain w...
                              ...                        
4804    el mariachi want play guitar carri famili trad...
4805    newlyw coupl honeymoon upend arriv respect sister
4806    sign seal deliv introduc dedic quartet civil s...
4807    ambiti new york attorney sam sent shanghai ass...
4808    ever sinc second grade first saw et extraterre...
Name: preprocessed_overview, Length: 4809, dtype: object

### 'cast' column

In [23]:
movies['cast'].head()
movies['cast'][0]

movies['cast'].isna().sum()

0

In [24]:
# to make all words lower-case
movies['cast']=movies['cast'].str.lower()

# Here we only need "names" from "key-words"
# As there are many "cast" in a movie , I'm just taking "5 front casts"
# we also need to remove " " As 'johnny','depp' & 'johnny depp' can be represent as different  
def casts_extractor(df):
    cast=[i["name"].replace(" ","-") for i in eval(df)]    # extract only "name"s from "keywords-column"
    return cast[:5]     # First 5 cast names
  
movies['cast']=movies['cast'].apply(casts_extractor)
movies['cast'][1]

['johnny-depp',
 'orlando-bloom',
 'keira-knightley',
 'stellan-skarsgård',
 'chow-yun-fat']

In [25]:
## Unique values 
get_unique_value(movies["cast"])

{'topher-grace',
 'brandon-hammond',
 'barret-oliver',
 'allegra-fulton',
 'ryan-mccluskey',
 'parker-riggs',
 'sherri-shepherd',
 'anna-faris',
 'jiang-wu',
 'fernanda-machado',
 'january-jones',
 'pat-hingle',
 'candace-evanofski',
 'farrah-fawcett',
 'glenn-hughes',
 'anushka-shetty',
 'chris-evans',
 'joan-allen',
 'joey-morgan',
 'steele-stebbins',
 'andrei-finti',
 'noomi-rapace',
 'owen-black',
 'cristina-contes',
 'ellie-chidzey',
 'shannen-doherty',
 'go-soo-hee',
 'jojo-wright',
 'steve-kahan',
 "mo'nique",
 'honorato-magaloni',
 'alison-armitage',
 'edie-mcclurg',
 'sammy-davis,-jr.',
 'heather-matarazzo',
 'paul-borghese',
 'angela-kinsey',
 'clancy-brown',
 'esther-wurmfeld',
 'talal-el-jordi',
 'shawn-ashmore',
 'aitana-sánchez-gijón',
 'm.l.-piyapas-bhirombhakdi',
 'brittany-curran',
 'michael-cortez',
 'suzuka-ohgo',
 'patrick-bauchau',
 'sarah-polley',
 'alison-raimondi',
 'raven-symoné',
 'phillip-alford',
 'castille-landon',
 'jennifer-prior',
 'park-overall',
 'seij

### 'crew' column

In [26]:
movies['crew'].head()
movies['crew'][0]
movies['crew'].isna().sum()


0

In [27]:
# to make all words lower-case
movies['crew']=movies['crew'].str.lower()

# Crews are important but it's lots of information to process & also not all crew members equally important for our task 
# So I'm taking only "director" & "producer"
# we also need to remove " " As 'johnny','depp' & 'johnny depp' can be represent as different  
def director_extractor(df):
    """ Extract Director name from Crew Members"""
    name=[i["name"].replace(" ","-") for i in eval(df) if i["job"]=="director"]    # extract only list of "director-names" 
    return [name[0] if len(name)!=0 else ""]  # if there are multiple "directors" return 1st && if no-director name listed then return "" 

movies['director']=movies['crew'].apply(director_extractor)

def producer_extractor(df):
    """ Extract Producer name from Crew Members"""
    name=[i["name"].replace(" ","-") for i in eval(df) if i["job"]=="producer"]    # extract only list of "producer-names" 
    #return [name[0] if len(name)!=0 else ""]  # if there are multiple "producers" return 1st && if no-producer name listed then return "" 
    return name

movies['producer']=movies['crew'].apply(producer_extractor)


## As we extracted valuable info. from "crew" column, so we can now drop that
movies.drop(['crew'],axis=1,inplace=True)

In [28]:
### In Some movies there are same "producer" as "director". So we have take one when they are same 
movies[['producer','director']].head()

movies["producer_director"]=movies["producer"]+movies["director"]
movies["producer_director"]

0              [james-cameron, jon-landau, james-cameron]
1       [jerry-bruckheimer, eric-mcleod, chad-oman, pe...
2       [barbara-broccoli, michael-g.-wilson, sam-mendes]
3       [charles-roven, christopher-nolan, emma-thomas...
4       [colin-wilson, jim-morris, lindsey-collins, an...
                              ...                        
4804    [robert-rodriguez, carlos-gallardo, robert-rod...
4805    [edward-burns, william-rexer, aaron-lubin, edw...
4806                           [harvey-kahn, scott-smith]
4807                                        [daniel-hsia]
4808                                   [brian-herzlinger]
Name: producer_director, Length: 4809, dtype: object

In [29]:
def add_director_producer(df):
    """ Remove duplicate value from director & producer columns """
    list_=[]
    [list_.append(i) for i in df if i not in list_]
    return list_

movies['producer_director']=movies['producer_director'].apply(add_director_producer)
movies['producer_director']


0                             [james-cameron, jon-landau]
1       [jerry-bruckheimer, eric-mcleod, chad-oman, pe...
2       [barbara-broccoli, michael-g.-wilson, sam-mendes]
3         [charles-roven, christopher-nolan, emma-thomas]
4       [colin-wilson, jim-morris, lindsey-collins, an...
                              ...                        
4804                  [robert-rodriguez, carlos-gallardo]
4805           [edward-burns, william-rexer, aaron-lubin]
4806                           [harvey-kahn, scott-smith]
4807                                        [daniel-hsia]
4808                                   [brian-herzlinger]
Name: producer_director, Length: 4809, dtype: object

### 'production_companies' column

In [30]:
movies['production_companies'].head()
movies['production_companies'][0]
movies['production_companies'].isna().sum()

0

In [31]:
# to make all words lower-case
movies['production_companies']=movies['production_companies'].str.lower()

# I did not know "production_companies" important or not but for me, some times I ignore movies thats are not belongs to known "production_companies"
# so I'm keeping this column you can skip it
# Here I'm taking only 2 front-line "production_companies" 
# we also need to remove " " As 'johnny','depp' & 'johnny depp' can be represent as different  
def production_companies_extractor(df):
    """ Extract production_companies name """
    name=[i["name"].replace(" ","-") for i in eval(df)]    # extract only "name"s from "keywords-column"
    return name[:2]   # extract Top2 

movies['production_companies']=movies['production_companies'].apply(production_companies_extractor)
movies['production_companies']

0       [ingenious-film-partners, twentieth-century-fo...
1         [walt-disney-pictures, jerry-bruckheimer-films]
2                             [columbia-pictures, danjaq]
3                      [legendary-pictures, warner-bros.]
4                                  [walt-disney-pictures]
                              ...                        
4804                                  [columbia-pictures]
4805                                                   []
4806    [front-street-pictures, muse-entertainment-ent...
4807                                                   []
4808         [rusty-bear-entertainment, lucky-crow-films]
Name: production_companies, Length: 4809, dtype: object

### 'original_language'

In [32]:
movies['original_language'].head()
movies['original_language'][0]
movies['original_language'].isna().sum()   # 0

movies['original_language'].value_counts()
# top 10 ==> en:4510 ; fr:70 ; es:32 ; zh:27 ; de:27 ; hi:19 ; ja:16 ; it:14 ; cn:12 ; ko:12

en    4510
fr      70
es      32
zh      27
de      27
hi      19
ja      16
it      14
cn      12
ko      12
ru      11
pt       9
da       7
sv       5
nl       4
fa       4
th       3
he       3
id       2
ar       2
ro       2
cs       2
ta       2
tr       1
hu       1
vi       1
nb       1
af       1
el       1
pl       1
ky       1
xx       1
te       1
ps       1
no       1
is       1
sl       1
Name: original_language, dtype: int64

### 'original_title'

In [33]:
movies['original_title'].head()
movies['original_title'][0]
movies['original_title'].isna().sum()  # 0

0

In [34]:
movies['original_title']==movies['title']

0       True
1       True
2       True
3       True
4       True
        ... 
4804    True
4805    True
4806    True
4807    True
4808    True
Length: 4809, dtype: bool

### 'popularity'

In [35]:
movies['popularity'].head()
movies['popularity'][0]
movies['popularity'].isna().sum()  # 0

movies['popularity'].describe()

count    4809.000000
mean       21.491664
std        31.803366
min         0.000000
25%         4.667230
50%        12.921594
75%        28.350529
max       875.581305
Name: popularity, dtype: float64

In [36]:
## Normalization
""" Note:
        standardized_df=(df-df.mean())/df.std()           # standardization produced most of -ve value (so i ignored)
        normalized_df=(df-df.min())/(df.max()-df.min())   # normalization produces range between 0-1
"""
#movies['normalized_popularity']=(movies['popularity']-movies['popularity'].mean())/movies['popularity'].std()
movies['normalized_popularity']=(movies['popularity']-movies['popularity'].min())/(movies['popularity'].max()-movies['popularity'].min())
movies['normalized_popularity'].describe()

count    4809.000000
mean        0.024546
std         0.036323
min         0.000000
25%         0.005330
50%         0.014758
75%         0.032379
max         1.000000
Name: normalized_popularity, dtype: float64

### 'production_countries' 

In [37]:
movies['production_countries' ].head()
movies['production_countries' ][4]
movies['production_countries' ].isna().sum()

0

In [38]:
def production_countries_extractor(df):
    production_countries=[i["name"].replace(" ","-") for i in eval(df)]    # extract only "name"s from "production_countries-column"
    return production_countries

movies["production_countries"]=movies["production_countries"].apply(production_countries_extractor)
movies["production_countries"]

0       [United-States-of-America, United-Kingdom]
1                       [United-States-of-America]
2       [United-Kingdom, United-States-of-America]
3                       [United-States-of-America]
4                       [United-States-of-America]
                           ...                    
4804            [Mexico, United-States-of-America]
4805                                            []
4806                    [United-States-of-America]
4807             [United-States-of-America, China]
4808                    [United-States-of-America]
Name: production_countries, Length: 4809, dtype: object

### 'release_date' 

In [39]:
movies['release_date'].head()
movies['release_date'].dtype     # "object"
movies['release_date'][0]
# movies['release_date'].isna().sum() # 1

# movies['release_date'].value_counts()


'2009-12-10'

In [40]:
### incomplete 

### 'revenue' 

In [41]:
movies['revenue'].head()
movies['revenue'][0]
movies['revenue'].isna().sum()

movies['revenue'].max(),movies['revenue'].min()    # (2787965087, 0)
movies['revenue'].describe()

count    4.809000e+03
mean     8.227511e+07
std      1.628379e+08
min      0.000000e+00
25%      0.000000e+00
50%      1.917000e+07
75%      9.291317e+07
max      2.787965e+09
Name: revenue, dtype: float64

In [42]:
movies['revenue'].value_counts() 

0            1430
7000000         6
8000000         6
100000000       5
10000000        5
             ... 
30987695        1
30016165        1
85498534        1
30426096        1
55003135        1
Name: revenue, Length: 3297, dtype: int64

### 'runtime' 

In [43]:
movies['runtime'].head()
movies['runtime'][0]
movies['runtime'].isna().sum()  # 2

movies['runtime'].value_counts() # 90.0:163 ; 100.0:149 ; 98.0:140
movies['runtime'].describe()

movies['runtime'].fillna(90.0,inplace=True)   # Nan value imputed with 90.0 as most of movies of that length

### 'spoken_languages' 

In [44]:
movies['spoken_languages'].head()
movies['spoken_languages'][0]
movies['spoken_languages'].isna().sum()

0

In [45]:
def spoken_languages_extractor(df):
    spoken_languages=[i["name"].replace(" ","-") for i in eval(df)]    # extract only "name"s from "spoken_languages-column"
    return spoken_languages

movies["spoken_languages"]=movies["spoken_languages"].apply(spoken_languages_extractor)
movies["spoken_languages"]

0                                    [English, Español]
1                                             [English]
2       [Français, English, Español, Italiano, Deutsch]
3                                             [English]
4                                             [English]
                             ...                       
4804                                          [Español]
4805                                                 []
4806                                          [English]
4807                                          [English]
4808                                          [English]
Name: spoken_languages, Length: 4809, dtype: object

### 'status'

In [46]:
movies['status'].head()
movies['status'][0]
movies['status'].isna().sum()

movies['status'].value_counts()  # Released:4801 ; Rumored:5 ; Post Production:3

Released           4801
Rumored               5
Post Production       3
Name: status, dtype: int64

### 'vote_average'

In [47]:
movies['vote_average'].head()
movies['vote_average'][0]
movies['vote_average'].isna().sum()

movies['vote_average'].max(),movies['vote_average'].min()  # (10.0, 0.0)
movies['vote_average'].value_counts() 
# 6.0:217 ; 6.5:217 ; 6.7:214 ; 6.3:207 ; 6.1:202

6.0    217
6.5    217
6.7    214
6.3    207
6.1    202
      ... 
9.5      1
2.6      1
2.7      1
0.5      1
9.3      1
Name: vote_average, Length: 71, dtype: int64

### 'vote_count'

In [48]:
movies['vote_count'].head()
movies['vote_count'][0]
movies['vote_count'].isna().sum()

movies['vote_count'].max(),movies['vote_count'].min()  # (13752, 0)
movies['vote_count'].value_counts()

0       62
1       53
2       46
4       43
3       41
        ..
1389     1
5487     1
1401     1
3452     1
2039     1
Name: vote_count, Length: 1609, dtype: int64

In [49]:
## Normalization
""" Note:
        standardized_df=(df-df.mean())/df.std()           # standardization produced most of -ve value (so i ignored)
        normalized_df=(df-df.min())/(df.max()-df.min())   # normalization produces range between 0-1
"""
#movies['normalized_vote_count']=(movies['vote_count']-movies['vote_count'].mean())/movies['vote_count'].std()
movies['normalized_vote_count']=(movies['vote_count']-movies['vote_count'].min())/(movies['vote_count'].max()-movies['vote_count'].min())
movies['normalized_vote_count'].describe()     

movies['normalized_vote_count'].max(),movies['normalized_vote_count'].min()   # (1.0, 0.0)

(1.0, 0.0)

### 'homepage'

In [50]:
movies['homepage'].head()
movies['homepage'][0]
movies['homepage'].isna().sum()  #3096

3096

### "tagline"

In [51]:
movies["tagline"].head()
movies["tagline"][0]
movies["tagline"].isna().sum() # 844

844

## Select Important features

In [52]:
movies.shape       # (4809, 29)
movies.columns
# All features
""" ['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language','original_title', 'overview', 'popularity',
'production_companies','production_countries', 'release_date', 'revenue', 'runtime','spoken_languages', 'status', 'tagline', 
    'title', 'vote_average','vote_count', 'movie_id', 'cast', 'preprocessed_overview', 'director',
       'producer', 'producer_director', 'normalized_popularity','normalized_vote_count', 'release_year']"""

selected_cols_v2=['budget','genres','keywords','original_language','production_companies','normalized_vote_count',
                  'production_countries','runtime','spoken_languages','title','vote_average','movie_id','cast',
                  'preprocessed_overview','director','producer','producer_director','normalized_popularity','overview']

In [53]:
Selected_data_v2=movies[selected_cols_v2]
Selected_data_v2.shape         # (4809, 19)
Selected_data_v2.isna().sum()  # 0
Selected_data_v2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   budget                 4809 non-null   int64  
 1   genres                 4809 non-null   object 
 2   keywords               4809 non-null   object 
 3   original_language      4809 non-null   object 
 4   production_companies   4809 non-null   object 
 5   normalized_vote_count  4809 non-null   float64
 6   production_countries   4809 non-null   object 
 7   runtime                4809 non-null   float64
 8   spoken_languages       4809 non-null   object 
 9   title                  4809 non-null   object 
 10  vote_average           4809 non-null   float64
 11  movie_id               4809 non-null   int64  
 12  cast                   4809 non-null   object 
 13  preprocessed_overview  4809 non-null   object 
 14  director               4809 non-null   object 
 15  prod

### Generalize "normalized_popularity","vote_average","normalized_vote_count"

In [54]:
# There are these type of measures about the movie (to make it much easier , i will combine all)
Selected_data_v2[["normalized_popularity","vote_average","normalized_vote_count"]]

## ("normalized_popularity"*"vote_average")+"normalized_vote_count"
Selected_data_v2["rating"]=round((Selected_data_v2["normalized_popularity"]*Selected_data_v2["normalized_vote_count"])+Selected_data_v2["vote_average"],1)
Selected_data_v2["rating"].min(),Selected_data_v2["rating"].max()  # (0.0, 10.0)
Selected_data_v2["rating"]

# Drop "normalized_popularity","vote_average","normalized_vote_count"
Selected_data_v2.drop(["normalized_popularity","vote_average","normalized_vote_count"],axis=1,inplace=True)

<ipython-input-54-7e18dad04e30>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Selected_data_v2["rating"]=round((Selected_data_v2["normalized_popularity"]*Selected_data_v2["normalized_vote_count"])+Selected_data_v2["vote_average"],1)
c:\Users\jitan\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### "budget"

In [55]:
Selected_data_v2["budget"].head()
Selected_data_v2["budget"].describe()

## we can convert it to Millions unit
Selected_data_v2["budget"]=Selected_data_v2["budget"]//1000000

## Renaming column to add Unit
Selected_data_v2.rename(columns={"budget":"budget (in Million)"},inplace=True)

Selected_data_v2.head()

<ipython-input-55-b2f4800deec4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Selected_data_v2["budget"]=Selected_data_v2["budget"]//1000000
c:\Users\jitan\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,budget (in Million),genres,keywords,original_language,production_companies,production_countries,runtime,spoken_languages,title,movie_id,cast,preprocessed_overview,director,producer,producer_director,overview,rating
0,237,"[action, adventure, fantasy, science-fiction]","[culture-clash, future, space-war, space-colon...",en,"[ingenious-film-partners, twentieth-century-fo...","[United-States-of-America, United-Kingdom]",162.0,"[English, Español]",Avatar,19995,"[sam-worthington, zoe-saldana, sigourney-weave...",22nd centuri parapleg marin dispatch moon pand...,[james-cameron],"[james-cameron, jon-landau]","[james-cameron, jon-landau]",in the 22nd century a paraplegic marine is dis...,7.3
1,300,"[adventure, fantasy, action]","[ocean, drug-abuse, exotic-island, east-india-...",en,"[walt-disney-pictures, jerry-bruckheimer-films]",[United-States-of-America],169.0,[English],Pirates of the Caribbean: At World's End,285,"[johnny-depp, orlando-bloom, keira-knightley, ...",captain barbossa long believ dead come back li...,[gore-verbinski],"[jerry-bruckheimer, eric-mcleod, chad-oman, pe...","[jerry-bruckheimer, eric-mcleod, chad-oman, pe...",captain barbossa long believed to be dead has ...,7.0
2,245,"[action, adventure, crime]","[spy, based-on-novel, secret-agent, sequel, mi...",en,"[columbia-pictures, danjaq]","[United-Kingdom, United-States-of-America]",148.0,"[Français, English, Español, Italiano, Deutsch]",Spectre,206647,"[daniel-craig, christoph-waltz, léa-seydoux, r...",cryptic messag bond past send trail uncov sini...,[sam-mendes],"[barbara-broccoli, michael-g.-wilson]","[barbara-broccoli, michael-g.-wilson, sam-mendes]",a cryptic message from bonds past sends him on...,6.3
3,250,"[action, crime, drama, thriller]","[dc-comics, crime-fighter, terrorist, secret-i...",en,"[legendary-pictures, warner-bros.]",[United-States-of-America],165.0,[English],The Dark Knight Rises,49026,"[christian-bale, michael-caine, gary-oldman, a...",follow death district attorney harvey dent bat...,[christopher-nolan],"[charles-roven, christopher-nolan, emma-thomas]","[charles-roven, christopher-nolan, emma-thomas]",following the death of district attorney harve...,7.7
4,260,"[action, adventure, science-fiction]","[based-on-novel, mars, medallion, space-travel...",en,[walt-disney-pictures],[United-States-of-America],132.0,[English],John Carter,49529,"[taylor-kitsch, lynn-collins, samantha-morton,...",john carter warweari former militari captain w...,[andrew-stanton],"[colin-wilson, jim-morris, lindsey-collins]","[colin-wilson, jim-morris, lindsey-collins, an...",john carter is a warweary former military capt...,6.1


In [56]:
### converts list to strings
Selected_data_v2['preprocessed_keywords']=Selected_data_v2['keywords'].apply(' '.join)   # converts list to strings
Selected_data_v2['preprocessed_genres']=Selected_data_v2['genres'].apply(' '.join)       # converts list to strings
Selected_data_v2['preprocessed_cast']=Selected_data_v2['cast'].apply(' '.join)           # converts list to strings
Selected_data_v2['preprocessed_director']=Selected_data_v2['director'].apply(' '.join)       # converts list to strings
Selected_data_v2['preprocessed_producer']=Selected_data_v2['producer'].apply(' '.join)       # converts list to strings
Selected_data_v2['preprocessed_spoken_languages']=Selected_data_v2['spoken_languages'].apply(' '.join)       # converts list to strings
Selected_data_v2['preprocessed_production_companies']=Selected_data_v2['production_companies'].apply(' '.join)       # converts list to strings
Selected_data_v2['preprocessed_production_countries']=Selected_data_v2['production_countries'].apply(' '.join)       # converts list to strings

## Dropinng all columns
Selected_data_v2.drop(['keywords','genres','cast','director','producer','spoken_languages','production_companies','production_countries'],axis=1,inplace=True)


<ipython-input-56-171df6e4ffef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Selected_data_v2['preprocessed_keywords']=Selected_data_v2['keywords'].apply(' '.join)   # converts list to strings
<ipython-input-56-171df6e4ffef>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Selected_data_v2['preprocessed_genres']=Selected_data_v2['genres'].apply(' '.join)       # converts list to strings
<ipython-input-56-171df6e4ffef>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [59]:
Selected_data_v2['preprocessed_producer_director']=Selected_data_v2['producer_director'].apply(' '.join)       # converts list to strings

## Dropinng all columns
Selected_data_v2.drop(['producer_director'],axis=1,inplace=True)


<ipython-input-59-e9ee2a3c066d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Selected_data_v2['preprocessed_producer_director']=Selected_data_v2['producer_director'].apply(' '.join)       # converts list to strings
c:\Users\jitan\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [63]:
Selected_data_v2.head(2)

Selected_data_v2.to_csv(r"D:\Projects\Content-Based_Movie_Recommendation_System-main\Backend\Data\Preprocessed_v2_data.csv")


In [58]:
#Selected_data_v2.loc[Selected_data_v2['preprocessed_genres'] == "action"]
#Selected_data_v2[Selected_data_v2['preprocessed_genres'].str.lower().str.contains("action drama")]  ## for genre matching
